# report

In [187]:
import pandas as pd
import os

In [83]:
file_list=['spy/'+i for i in os.listdir('spy')]

In [84]:
sample = pd.read_csv(file_list[0],skipinitialspace=True)

In [328]:
import pandas as pd
import os

def contract_selct(
    selct_type: str = 'main',
    contract_type: str = 'C',
    data : str = 'spy',
    date: str = None,
    main: pd.DataFrame = None,
    maxDTE: int = 20,
    minDTE: int = 10,
    maxDistance: float = None,
    minDistance: float = None,
    maxTheta: float = None,
    minTheta: float = None,
    moneyness : str = 'out') ->pd.DataFrame:
    
    """
    return a dataframe containing the information of selected contracts
    
    input:
    selct_type: "main" or "hedge", declare what type of contracts we are searching, whether the main contracts we are shorting, or the contracts that we used to hedge
    contract_type: 'C' or 'P'
    date: str in form of "yyyymmdd", declare the date
    data : 'spy' or 'spx'
    main: None when selct_type = 'main', a dataframe containing the main contract when selct_type = 'hedge'
    maxDTE: max DTE
    minDTE: min DTE
    maxDistance: max distance of strike from underlying in percentage
    minDistance: min distance of strike from underlying in percentage
    maxTheta: max Theta
    minTheta: min Theta
    moneyness: 'out' or 'in'
    """
    
    if maxDistance == None: maxDistance = 1000
    if minDistance == None: minDistance = 0
    if maxTheta == None: maxTheta = 0
    if minTheta == None: minTheta = -100
    
    contract_pd = pd.DataFrame()
    
    if selct_type == 'main':
        contract_pd =  pd.read_csv(data+'_cleaned/'+data+'_eod_'+date[:6]+'.csv', 
                           index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
        contract_pd = contract_pd.drop(columns='Unnamed: 0')
        contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
        contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
                & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]
        if contract_type == 'C':
            contract_pd = contract_pd.drop(columns=['[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]'])
            if moneyness == 'out':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
            if moneyness == 'in':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
        if contract_type == 'P':
            contract_pd = contract_pd.drop(columns=['[C_BID]', '[C_ASK]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]'])
            if moneyness == 'out':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
            if moneyness == 'in':
                contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
                contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
    
        contract_pd['spread_pct'] = (contract_pd['['+contract_type+'_ASK]']-contract_pd['['+contract_type+'_BID]'])*2/(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])
        # Set inverted quote spread to zero
        contract_pd['spread_pct'][contract_pd['spread_pct'] < 0] = 0
        
        # filter out contract with very small ask
        contract_pd = contract_pd[contract_pd['['+contract_type+'_ASK]'] >= contract_pd['['+contract_type+'_ASK]'].mean()]
        contract_pd = contract_pd[contract_pd['['+contract_type+'_ASK]'] >= 0.3]
        
        for i in ['['+contract_type+'_DELTA]', '['+contract_type+'_GAMMA]', '['+contract_type+'_VEGA]', '['+contract_type+'_THETA]']:
            contract_pd['price_to_'+i]=(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])/(2*contract_pd[i].abs())

        # utility function to select main contract    
        if data == 'spx': contract_pd['utility'] = 5000*contract_pd['spread_pct'] - 0.5*contract_pd['price_to_['+contract_type+'_DELTA]'] -0.005*contract_pd['price_to_['+contract_type+'_GAMMA]'] - contract_pd['price_to_['+contract_type+'_VEGA]'] + 50* contract_pd['price_to_['+contract_type+'_THETA]']
        if data == 'spy': contract_pd['utility'] = 1000*contract_pd['spread_pct'] - 3*contract_pd['price_to_['+contract_type+'_DELTA]'] -contract_pd['price_to_['+contract_type+'_GAMMA]'] - 3* contract_pd['price_to_['+contract_type+'_VEGA]'] + 50* contract_pd['price_to_['+contract_type+'_THETA]']
        
        contract_pd = contract_pd.drop(columns=['spread_pct', 'price_to_['+contract_type+'_DELTA]', 'price_to_['+contract_type+'_GAMMA]', 'price_to_['+contract_type+'_VEGA]', 'price_to_['+contract_type+'_THETA]'])

    return contract_pd.sort_values(by=['utility']).head(3)
    # return contract_pd

In [329]:
contract_selct(date= '20200910', minDistance=0.08)

[UNDERLYING_LAST]  [DTE]  [C_DELTA]  [C_GAMMA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                   
2020-09-10   2020-09-30                333.92   20.0    0.04383    0.00552   
             2020-09-30                333.92   20.0    0.04856    0.00606   
             2020-09-30                333.92   20.0    0.05339    0.00653   

                            [C_VEGA]  [C_THETA]  [C_BID]  [C_ASK]  [STRIKE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                    
2020-09-10   2020-09-30      0.07469   -0.03374     0.28     0.30     363.0   
             2020-09-30      0.08088   -0.03625     0.32     0.34     362.0   
             2020-09-30      0.08780   -0.03899     0.35     0.38     361.0   

                            [STRIKE_DISTANCE]  [STRIKE_DISTANCE_PCT]  \
[QUOTE_DATE] [EXPIRE_DATE]                                             
2020-09-10   2020-09-30                  29.1                  0.087   
             2020-09-30                  28.1                  0.084   
             2020-09-30                  27.1                  0.081   

                               utility  
[QUOTE_DATE] [EXPIRE_DATE]              
2020-09-10   2020-09-30     414.688687  
             2020-09-30     428.695523  
             2020-09-30     461.383666

In [322]:
data= 'spy'
date= '20130910'
maxDTE: int = 30,
minDTE: int = 10,
maxDistance: float = 0.2,
minDistance: float = 0.01,
maxTheta: float = -0.0,
minTheta: float = -1.2,
contract_type: str = 'P'
moneyness = 'out'

contract_pd =  pd.read_csv(data+'_cleaned/'+data+'_eod_'+date[:6]+'.csv', 
                   index_col=['[QUOTE_DATE]','[EXPIRE_DATE]'], skipinitialspace=True)
contract_pd = contract_pd.drop(columns='Unnamed: 0')
contract_pd = contract_pd.iloc[contract_pd.index.get_level_values('[QUOTE_DATE]')==date[:4]+'-'+date[4:6]+'-'+date[6:]]
contract_pd = contract_pd.loc[(contract_pd['[DTE]']>=minDTE) & (contract_pd['[DTE]']<=maxDTE) 
                & (contract_pd['[STRIKE_DISTANCE_PCT]']>=minDistance) & (contract_pd['[STRIKE_DISTANCE_PCT]']<=maxDistance)]

if contract_type == 'C':
    contract_pd = contract_pd.drop(columns=['[P_BID]', '[P_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]'])
    if moneyness == 'out':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
    if moneyness == 'in':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[C_THETA]'] >= minTheta) & (contract_pd['[C_THETA]'] <= maxTheta)]
if contract_type == 'P':
    contract_pd = contract_pd.drop(columns=['[C_BID]', '[C_ASK]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]'])
    if moneyness == 'out':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] <= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]
    if moneyness == 'in':
        contract_pd = contract_pd.loc[contract_pd['[STRIKE]'] >= contract_pd['[UNDERLYING_LAST]']]
        contract_pd = contract_pd.loc[(contract_pd['[P_THETA]'] >= minTheta) & (contract_pd['[P_THETA]'] <= maxTheta)]



In [323]:
contract_pd['spread_pct'] = (contract_pd['['+contract_type+'_ASK]']-contract_pd['['+contract_type+'_BID]'])*2/(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])
contract_pd['spread_pct'][contract_pd['spread_pct'] < 0] = 0
contract_pd = contract_pd[contract_pd['['+contract_type+'_ASK]'] >= contract_pd['['+contract_type+'_ASK]'].mean()]

for i in ['['+contract_type+'_DELTA]', '['+contract_type+'_GAMMA]', '['+contract_type+'_VEGA]', '['+contract_type+'_THETA]']:
    contract_pd['price_to_'+i]=(contract_pd['['+contract_type+'_BID]']+contract_pd['['+contract_type+'_ASK]'])/(2*contract_pd[i].abs())

# utility function to select main contract    
contract_pd['utility'] = 1000*contract_pd['spread_pct'] - 3*contract_pd['price_to_['+contract_type+'_DELTA]'] -contract_pd['price_to_['+contract_type+'_GAMMA]'] - 3* contract_pd['price_to_['+contract_type+'_VEGA]'] + 50* contract_pd['price_to_['+contract_type+'_THETA]']

# contract_pd = contract_pd.drop(columns=['spread_pct', 'price_to_['+contract_type+'_DELTA]', 'price_to_['+contract_type+'_GAMMA]', 'price_to_['+contract_type+'_VEGA]', 'price_to_['+contract_type+'_THETA]'])

contract_pd.sort_values(by=['utility'])

[UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                
2013-09-10   2013-09-20                168.88   10.0     164.0     0.37   
             2013-09-20                168.88   10.0     165.0     0.51   
             2013-09-20                168.88   10.0     166.0     0.72   
             2013-09-27                168.88   17.0     161.0     0.33   
             2013-09-27                168.88   17.0     161.5     0.37   
             2013-09-27                168.88   17.0     162.0     0.42   
             2013-09-27                168.88   17.0     162.5     0.48   
             2013-09-27                168.88   17.0     163.0     0.53   
             2013-09-20                168.88   10.0     167.0     1.00   
             2013-09-30                168.88   20.0     160.0     0.31   
             2013-09-27                168.88   17.0     163.5     0.60   
             2013-09-30                168.88   20.0     161.0     0.40   
             2013-10-04                168.88   24.0     158.0     0.32   
             2013-09-30                168.88   20.0     162.0     0.49   
             2013-09-27                168.88   17.0     164.0     0.68   
             2013-10-04                168.88   24.0     159.0     0.36   
             2013-09-27                168.88   17.0     164.5     0.77   
             2013-10-04                168.88   24.0     160.0     0.46   
             2013-10-04                168.88   24.0     160.5     0.50   
             2013-09-30                168.88   20.0     163.0     0.62   
             2013-10-04                168.88   24.0     161.0     0.56   
             2013-09-27                168.88   17.0     165.0     0.87   
             2013-10-04                168.88   24.0     161.5     0.61   
             2013-09-27                168.88   17.0     165.5     0.99   
             2013-09-30                168.88   20.0     164.0     0.78   
             2013-10-04                168.88   24.0     162.0     0.67   
             2013-09-27                168.88   17.0     166.0     1.11   
             2013-10-04                168.88   24.0     162.5     0.75   
             2013-10-04                168.88   24.0     163.0     0.83   
             2013-09-30                168.88   20.0     165.0     0.98   
             2013-09-27                168.88   17.0     166.5     1.24   
             2013-10-04                168.88   24.0     163.5     0.91   
             2013-10-04                168.88   24.0     164.0     1.01   
             2013-09-27                168.88   17.0     167.0     1.41   
             2013-09-30                168.88   20.0     166.0     1.23   
             2013-10-04                168.88   24.0     164.5     1.13   
             2013-10-04                168.88   24.0     165.0     1.23   
             2013-10-04                168.88   24.0     165.5     1.35   
             2013-09-30                168.88   20.0     167.0     1.55   
             2013-10-04                168.88   24.0     166.0     1.50   
             2013-10-04                168.88   24.0     166.5     1.63   
             2013-10-04                168.88   24.0     167.0     1.81   

                            [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                            
2013-09-10   2013-09-20        0.39   -0.15973    0.05383   0.07136   
             2013-09-20        0.53   -0.21042    0.06704   0.08453   
             2013-09-20        0.73   -0.27970    0.08064   0.09813   
             2013-09-27        0.34   -0.11133    0.03095   0.07103   
             2013-09-27        0.38   -0.12381    0.03407   0.07630   
             2013-09-27        0.42   -0.13815    0.03748   0.08271   
             2013-09-27        0.47   -0.15492    0.04103   0.08938   
             2013-09-27        0.54   -0.17316    0.04489   0.09518   
             2013

In [296]:
contract_pd['spread_pct'][contract_pd['spread_pct'] < 0]=0

In [297]:
sum(contract_pd['spread_pct'] < 0)

0

In [313]:
contract_pd.to_csv('test.csv')